In [1]:
import pandas as pd
import numpy as np
import os
import re
import json

In [2]:

bdir=os.path.join(os.getcwd(),"results")
sdir=os.path.join(os.getcwd(),"results_csv")

def gen_df_from_json_dir(case,resdir=bdir):
    
    print("          building df conntainer ")
    data_path=os.path.join(resdir,case)
    start_file_path=os.path.join(data_path,"start_.json")
    
    with open(start_file_path,'r') as f:
        
        start_dict_=json.load(f)
        
    df=pd.DataFrame(columns=start_dict_.keys())

    files = [i for i in os.listdir(os.path.join(resdir,res)) if i!='data.json' and i!="start_.json"]

    
    print("           renaming files so that they are chronologically ordered")
    
    
    reg=r"epoch_[0-9]+.json"
    
    for file in files:
        if re.match(reg,file) is not None:
            
            new_name=file.replace(".json","_batch_0_t_999999999.json")
            
            os.rename(os.path.join(data_path,file), os.path.join(data_path,new_name))
    
    print("           reloading file names")
    
    files = [i for i in os.listdir(data_path) if i!='data.json' and i!="start_.json" and ('WIND' not in i)]
    files=sorted(files)
    
    print("           pumping data into df")
    

    for count,file in enumerate(files):
        print("\r               Reading %i / %i "%(count,len(files)), end='', flush=True)

        with open(os.path.join(data_path,file),'r') as f:
            data_dict_=json.load(f)
        df=df.append(data_dict_,ignore_index=True)
        
        
    print("            recovering parameters ")
    data_file_path=os.path.join(data_path,"data.json")
    
    with open(data_file_path,'r') as f:
        data_dict_=json.load(f)
    
    keep_cols=("used_logged_v_in_model",
              'fit_on_v',
              'base_lr',
              "n_epochs",
              "nsecs")
    for key_ in keep_cols:
        df.insert(len(df.columns), key_, data_dict_[key_])
        
    return df 



In [3]:
selected_files=[file for file in os.listdir(bdir)]
failed=[]



for a,file in enumerate(selected_files):

    res=file
    
    csvname=res+".csv"
    
    if csvname not in os.listdir(sdir):

        print(file,"  %i/%i"%(a,len(selected_files)))
    #         resdir=os.path.join(os.getcwd(),"results")

    #         dirpath=os.path.join(resdir,res)

        try:
            df=gen_df_from_json_dir(res)
            df.to_csv(os.path.join(sdir,res+".csv"))
        except:
            print('FAILEEEEED')
            failed.append(file)
    else:
        print("Already here, skiiipiiing...")
        
print("Failed to proceed : ",failed)

AVION_14_sept_fit_v_False_lr_0.005_ns_all   0/27
          building df conntainer 
           renaming files so that they are chronologically ordered
           reloading file names
           pumping data into df
               Reading 5 / 6             recovering parameters 
AVION_14_sept_fit_v_False_lr_0.005_ns_1.0   1/27
          building df conntainer 
           renaming files so that they are chronologically ordered
           reloading file names
           pumping data into df
               Reading 51 / 52             recovering parameters 
AVION_14_sept_fit_v_False_lr_0.0005_ns_1.0   2/27
          building df conntainer 
           renaming files so that they are chronologically ordered
           reloading file names
           pumping data into df
               Reading 51 / 52             recovering parameters 
AVION_14_sept_fit_v_False_lr_0.05_ns_-1.0   3/27
          building df conntainer 
           renaming files so that they are chronologically ordered
           

In [4]:
res


'AVION_14_sept_fit_v_False_lr_c_ns_1.0'